In [ ]:
# Vereiste imports

import io, PyPDF4, os, fitz, sys, re
from PIL import Image,ImageOps

In [ ]:
# Specificeer map die de PDFs bevat

directory = "/path/to/directory"

In [ ]:
def extract_images(input_pdf):

    checkXO = r"/Type(?= */XObject)"       # finds "/Type/XObject"   
    checkIM = r"/Subtype(?= */Image)"      # finds "/Subtype/Image"

    docs = fitz.open(input_pdf)
    imgcount = 0
    lenXREF = docs._getXrefLength()         # number of objects - do not use entry 0!

    # display some file info
    print("file: %s, pages: %s, objects: %s" % (input_pdf, len(docs), lenXREF-1))

    for i in range(1, lenXREF):            # scan through all objects
        text = docs._getXrefString(i)     # string defining the object
        isXObject = re.search(checkXO, text)    # tests for XObject
        isImage   = re.search(checkIM, text)    # tests for Image
        if not isXObject or not isImage:   # not an image object if not both True
            continue
        imgcount += 1
        pix = fitz.Pixmap(docs, i)          # make pixmap from image
        try:
            pix0 = fitz.Pixmap(fitz.csRGB, pix)
            pix0.writePNG((directory+"/imgs/"+filename+"img-%s.png") % (i,))
        except Exception:
            print("unable to extract image for", input_pdf)
        pix0 = None                    # free Pixmap resources
        pix = None                         # free Pixmap resources

    print("extracted images", imgcount)

In [ ]:
# Wanneer de map met output nog niet bestaat wordt deze aangemaakt

if not os.path.exists(directory+"/imgs/"):
    os.makedirs(directory+"/imgs/")

# Run de code voor elk PDF-bestand in de map    
    
for filename in os.listdir(directory):
    if filename.endswith((".pdf", ".PDF")):
        extract_images(directory+"/"+filename)